The (Classical) Hough Transform
===============================

In [1]:
import bqplot.pyplot as plt
import bqplot as bq
import numpy as np
import ipywidgets as widgets

The _classical_ Hough transform is a feature extraction technique used to find a subset of geometrical shapes within an image subject to imperfections. 

Lines
-----

In polar space, a line may be parametrised in *Hesse normal form* by the length $r$ of the normal to the origin $\vec{r}$, and the angle $\theta$ of $\vec{r}$ with respect to the origin:
$$
r(\theta) = x_i\cos(\theta) + y_i\sin(\theta)\,.
$$
For any point $(x, y)$ on this line, $r$ and $\theta$ are *constant*.

In [2]:
x = np.linspace(-200, 200)
    
plt.figure(max_aspect_ratio=1, min_aspect_ratio=1)
line = plt.plot(x=x, y=x*0, labels=['Line'], 
                options={'y': {'min': x.min(), 'max': x.max()}, 
                         'x': {'min': x.min(), 'max': x.max()}})
normal = plt.plot(x, x*0, 'r--', labels=['Normal'])
label = plt.label(text=[],x=[],y=[], colors=['orange'])
plt.legend()
plt.show()

In [3]:
x0 = y0 = x.min()
r_label = 200

@widgets.interact
def draw(m=widgets.FloatLogSlider(min=-2, max=2, description="-m"), c=(-400.0, 400.0)):
    m = -m
    y = m*x + c
    theta = np.pi/2 - np.arctan(-m)
    r = c / (np.sin(theta)-m*np.cos(theta))
    line.y = y; normal.y = (-1/m)*(x-x0) + y0
    label.y = [r_label*np.sin(theta) + y0]; label.x = [r_label*np.cos(theta) + x0]
    label.text = [f"(r={r:.1f}, θ={np.degrees(theta):.1f})"]

interactive(children=(FloatLogSlider(value=1.0, description='-m', max=2.0, min=-2.0), FloatSlider(value=0.0, d…

An example
----------
Let's define two lines

In [4]:
y1 = 3*x + 30
y2 = 1.5*x + 40

In [5]:
plt.figure() 
plt.plot(x, y1)
plt.plot(x, y2)
plt.show()


In the context of image analysis, the cartesian coordinates $(x_i,y_i)$ are known, whilst $(r,\theta)$ are the parameters to be solved for. Given that for all points on a straight line, $r$ and $\theta$ remain constant, it follows that there exist many lines upon which $(x_i, y_i)$ lies, and thus each $(x_i, y_i)$ in cartesian space maps to _curves_ in the polar Hough parameter space. Hence, points which are colinear in cartesian space (i.e. lie on the same line) will map to curves in Hough space which intersect at some $(r, \theta)$.


The transform is implemented by transforming each $(x_i, y_i)$ into Hough parameter space, and then incrementing an accumulator each $(r_i, \theta_i)$ along a discretisation of the $(r,\theta)$ curve. Identifying peaks in the resulting $(r,\theta)$ histogram indicates the presence of a line with the given parameters:

[^1]: http://people.scs.carleton.ca/~c_shu/Courses/comp4900d/notes/lect10_hough.pdf

![Quantisation in Hough Space.](quantisation.png)

### Determine $\theta$ and $r$ arrays from known $(x_i,y_i)$ points
Let's then determine a subset of the valid $(r,\theta)$ parameters for the first line (in discrete space).

In [6]:
theta1 = np.tile(np.linspace(0, 2*np.pi, 200).reshape(-1,1), len(x))
r1 = x*np.cos(theta1) + y1*np.sin(theta1)

### Plotting the parameter space
The parameter space $\{\,r,\theta\,\}$ can then be plotted as heatmap.

In [7]:
hist_2d, r1_edges, theta1_edges = np.histogram2d(r1.ravel(),  theta1.ravel(), bins=100)

In [8]:
plt.figure()
plt.heatmap(hist_2d, x=r1_edges, y=np.degrees(theta1_edges), cmap="viridis", tooltip=bq.Tooltip(fields=['midpoint']))
plt.xlabel("r")
plt.ylabel("theta")
plt.show()

### Isolating shapes
Let's extract the top four parameters by frequency.

In [9]:
max_indices = np.vstack(np.unravel_index(np.argsort(hist_2d, axis=None), hist_2d.shape)).T
maxima = max_indices[-1:-5:-1]

And then plot the fit line

In [10]:
i_r1, j_theta1 = maxima[0]
r1_fit = r1_edges[i_r1]
theta1_fit = theta1_edges[j_theta1]

In [11]:
plt.figure()
plt.plot(x, (r1_fit - x*np.cos(theta1_fit))/np.sin(theta1_fit), "r--", labels=['Fit'])
plt.plot(x, y1, labels=['True'])
plt.legend()
plt.show()

Evidently we've isolated the line by its points.

Lines in 3D
-----------
In three dimensions, there are now three parameters $(r,\theta,\phi)$ by which a line of the form $\vec{a}+t\vec{b}$ may be represented, where
$$
\vec{b}=\begin{pmatrix}b_x \\ b_y \\ b_z\end{pmatrix}=\begin{pmatrix}\cos\phi\cos\theta\\\sin\phi\cos\theta\\\sin\theta\end{pmatrix}\,.
$$

However, since two _antiparallel_ lines (i.e. lines which are parallel but traverse in opposite directions) describe the _same line_ (with a negative $t$), $\vec{b}$ must be confined to a half space, such that one cannot represent the same vector by reversing the signs of both $t$ and $b$. To do this, one may restrict the angles $\theta$ and $\phi$ to $0\leq\theta\leq\frac{\pi}{2}$ and $-\pi\leq\theta\leq\pi$ respectively.  
There will also be antiparallel vectors in the $(x,y)$ plane if $b_z=0$. These redundancies may be removed by constraining $b_y \geq 0$ if $b_z=0$, and $b_x = 1$ if $b_y=b_z=0$.

An arbitrary anchor point $\vec{a}$ in 3D cartesian space may be represented by three parameters. Yet one of these parameters is redundant, as we can choose _any arbitrary plane_ and define the anchor point as the _plane coordinates_ $(x^\prime,y^\prime)$ through which our line passes. 

![Spherical polar representation and transformed xy plane](3d-xform.png)

[^1]: https://www.ipol.im/pub/art/2017/208/article.pdf